<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/openai_agents_sdk/deep_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
google_api_key = userdata.get('GOOGLE_API_KEY')
openai_api_key=userdata.get('OPENAI_API_KEY')
#sendgrid_api_key=userdata.get('SENDGRID_API_KEY')

In [2]:
import os
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["GOOGLE_API_KEY"] = google_api_key
#os.environ["SENDGRID_API_KEY"] = sendgrid_api_key

In [4]:
!pip install openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 734.6/734.6 kB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0


In [9]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
#import sendgrid
import os
#from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown

In [10]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [11]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of June 2025, several AI agent frameworks have emerged, each offering unique capabilities for developing intelligent systems:

- **LangGraph**: An extension of LangChain, LangGraph introduces a graph-based system for managing agent workflows. It supports complex decision-making processes with features like branching, looping, and asynchronous task execution, making it suitable for applications such as loan processing and workflow automation. ([phyniks.com](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai))

- **AutoGen**: Developed by Microsoft, AutoGen focuses on multi-agent systems and code automation. It enables agents to communicate in natural language, facilitating tasks like planning, development, and review. AutoGen includes built-in agents and a graphical interface for prototyping, aiming to simplify the creation of collaborative AI agents. ([medium.com](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai))

- **CrewAI**: CrewAI emphasizes role-based task execution, assigning specific roles to agents to complete complex workflows. This approach mimics real-world team dynamics, enhancing specialization and task division, and is particularly effective in data science and content creation projects. ([odsc.medium.com](https://odsc.medium.com/top-10-open-source-ai-agent-frameworks-to-know-in-2025-c739854ec859?utm_source=openai))

- **Eliza**: Eliza is an open-source, Web3-friendly AI agent operating system that integrates seamlessly with blockchain applications. It allows for the deployment of AI agents capable of interacting with smart contracts and blockchain data, expanding the scope of AI agents into decentralized environments. ([arxiv.org](https://arxiv.org/abs/2501.06781?utm_source=openai))

- **AutoAgent**: AutoAgent is a fully automated, zero-code framework for LLM agents, enabling users to create and deploy AI agents through natural language instructions. It comprises components like an actionable engine and a self-managing file system, facilitating efficient agent development without coding requirements. ([arxiv.org](https://arxiv.org/abs/2502.05957?utm_source=openai))

These frameworks reflect the rapid advancements in AI agent development, offering diverse tools for building intelligent, autonomous systems across various applications. 

In [12]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 2

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# Use Pydantic to define the Schema of our response - this is known as "Structured Outputs"
# With massive thanks to student Wes C. for discovering and fixing a nasty bug with this!

class WebSearchItem(BaseModel):
    reason: str = Field(description="Your reasoning for why this search is important to the query.")

    query: str = Field(description="The search term to use for the web search.")


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of web searches to perform to best answer the query.")


planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [13]:
message = "Latest AI Agent frameworks in 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    print(result.final_output)

searches=[WebSearchItem(reason='To find recent developments in AI agent frameworks specifically for the year 2025.', query='latest AI agent frameworks 2025'), WebSearchItem(reason='To explore existing frameworks and their advancements, focusing on AI agents.', query='AI agent framework innovations 2025')]


In [15]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be lengthy and detailed. Aim "
    "for 5-10 pages of content, at least 1000 words."
)


class ReportData(BaseModel):
    short_summary: str = Field(description="A short 2-3 sentence summary of the findings.")

    markdown_report: str = Field(description="The final report")

    follow_up_questions: list[str] = Field(description="Suggested topics to research further")


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

In [16]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

In [17]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

In [18]:
query ="Latest AI Agent frameworks in 2025"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    #await send_email(report)
    print("Hooray!")

Starting research...
Planning searches...
Will perform 2 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Hooray!


In [23]:
Markdown(report.markdown_report)

# Latest AI Agent Frameworks in 2025: A Comprehensive Overview

## Table of Contents

1. [Introduction](#introduction)  
2. [Key AI Agent Frameworks in 2025](#key-ai-agent-frameworks-in-2025)  
   - LangGraph  
   - AutoGen  
   - Eliza  
   - OpenAI Operator  
   - Model Context Protocol (MCP)  
3. [Emerging Trends in AI Agent Technologies](#emerging-trends-in-ai-agent-technologies)  
   - Agentic AI Systems  
   - Task-Specific AI Agents  
   - Multi-Agent Collaboration  
   - Integration with IoT and Personal Devices  
   - Advancements in AI Reasoning  
   - AI-Augmented Decision-Making  
   - Edge AI  
   - AI-Powered Cybersecurity  
   - Green AI  
   - No-Code and Low-Code AI Builders  
4. [Conclusion](#conclusion)  
5. [References](#references)  

## Introduction  
The field of artificial intelligence is experiencing unprecedented growth, particularly in the development of AI agents—intelligent systems capable of autonomously executing tasks and interacting with their environment. As of 2025, several new frameworks have emerged which are reshaping how these AI agents are built and deployed. This report aims to provide an overview of the latest frameworks and the trends driving the evolution of AI agent technology.  

## Key AI Agent Frameworks in 2025  

### LangGraph  
LangGraph represents an innovative evolution of the LangChain framework, introducing a graph-based architecture that facilitates sophisticated agent workflows. This framework enables agents to navigate complex decision-making tasks by supporting dynamic branching, looping, and conditional flows. Applications benefiting from LangGraph include those requiring intricate logical processes, such as strategic business planning and advanced gaming systems.  

### AutoGen  
Developed by Microsoft, AutoGen emphasizes collaborative multi-agent interactions. Utilizing natural language processing, it enables agents to communicate and coordinate tasks seamlessly. Within this framework, built-in agents can work together in real-time, allowing for the rapid prototyping of complex AI systems. This collaboration not only speeds up development but also enhances the performance of deployed solutions across various sectors, including customer service and project management.  

### Eliza  
Eliza stands out as a Web3-friendly, open-source framework that integrates deeply with blockchain technologies. By allowing developers to create AI agents that can engage with smart contracts and access blockchain data, Eliza broadens the potential for decentralized applications (DApps). This integration is crucial for industries transitioning to blockchain solutions, enabling more automated and trustworthy transactions and communications.  

### OpenAI Operator  
Launched in early 2025, the OpenAI Operator employs advanced models to automate web interactions. This AI agent is designed to autonomously perform tasks such as filling out forms and scheduling appointments. Its capability to interact with web interfaces marks a significant step forward in automating routine online activities, enhancing productivity, and enabling users to focus on core tasks.  

### Model Context Protocol (MCP)  
Introduced by Anthropic, the Model Context Protocol (MCP) serves to standardize data access across disparate AI systems. By promoting interoperability and simplifying data integration, MCP allows AI agents to function more robustly while ensuring they can be easily adapted to various applications. This is crucial for the development of sophisticated AI solutions that require access to diverse datasets.  

## Emerging Trends in AI Agent Technologies  
The evolution of AI agent frameworks is accompanied by significant trends that highlight their increasing autonomy, specialization, and integration. These trends include:  

### Agentic AI Systems  
The emergence of autonomous AI agents capable of executing complex sequences of actions with minimal human oversight is becoming commonplace. These systems leverage advanced machine learning techniques to self-improve, leading to reductions in operational times and costs across knowledge-intensive operations.  

### Task-Specific AI Agents  
A marked focus on developing tailored AI agents for specific domains—such as customer support, financial planning, and sales forecasting—is gaining momentum. This specialization enables quicker training and deployment, resulting in higher performance and effectiveness when addressing distinct industry needs.  

### Multi-Agent Collaboration  
The paradigm is shifting from single-agent reliance to ecosystems of collaborative agents. These agents can communicate and share information, working together to complete intricate tasks and enhancing efficiency in enterprise workflows. This cooperation is vital for industries facing complex challenges requiring multiple agent contributions.  

### Integration with IoT and Personal Devices  
AI agents are being increasingly integrated with the Internet of Things (IoT), resulting in a unified communication framework among devices. This integration allows for enhanced automation and facilitates real-time decision-making in settings such as smart homes, cities, and offices.  

### Advancements in AI Reasoning  
AI agents are now improving their reasoning capabilities, progressing beyond basic pattern recognition to understanding causal relationships. This enables them to make logical inferences and solve problems systematically, allowing for thorough decision-making and enhancing their ability to adapt to new challenges.  

### AI-Augmented Decision-Making  
AI is transitioning into a co-pilot role in decision-making processes across various sectors. Agents are now expected to provide actionable insights and analytics that improve the accuracy and efficiency of tasks ranging from financial forecasts to medical diagnoses.  

### Edge AI  
The rise of Edge AI is proving crucial as processing occurs closer to data sources, enabling real-time operations. This is especially relevant in applications involving autonomous vehicles and drones, where immediacy can improve outcomes dramatically.  

### AI-Powered Cybersecurity  
Intelligent agents are becoming pivotal in cybersecurity, continuously monitoring systems and predicting threats. This proactive approach ensures quicker prevention and response times, thereby strengthening the security of digital ecosystems.  

### Green AI  
Sustainability is becoming a key focus for AI development. Solutions that prioritize energy efficiency—termed Green AI—are crucial as the industry seeks to reduce the carbon footprint associated with data-heavy applications without compromising performance.  

### No-Code and Low-Code AI Builders  
The advent of no-code and low-code platforms democratizes AI development, allowing individuals without deep programming knowledge to create AI agents. Using intuitive interfaces, these tools significantly lower the barriers to entry, fostering innovation across startups, educational sectors, and small to medium enterprises.  

## Conclusion  
As we navigate through 2025, the landscape of AI agent frameworks continues to progress rapidly. With frameworks like LangGraph, AutoGen, and Eliza, developers now have access to powerful tools that enhance collaboration, decision-making, and integration across diverse environments. The trends identified highlight a shift towards more autonomous, specialized, and sustainable AI solutions that promise to transform industries and augment capabilities globally. With these advancements, AI agents are poised to play a pivotal role in shaping future technologies and applications.

## References  
- [Phyniks - AI Agent Frameworks in 2025](https://phyniks.com/blog/top-7-agentic-ai-frameworks-in-2025?utm_source=openai)  
- [Medium - AutoGen and Collaborations](https://medium.com/%40elisowski/top-ai-agent-frameworks-in-2025-9bcedab2e239?utm_source=openai)  
- [arXiv - Eliza Framework](https://arxiv.org/abs/2501.06781?utm_source=openai)  
- [Wikipedia - OpenAI Operator](https://en.wikipedia.org/wiki/OpenAI_Operator?utm_source=openai)  
- [Wikipedia - MCP Protocol](https://en.wikipedia.org/wiki/Model_Context_Protocol?utm_source=openai)  
- [Zynova - AI Trends 2025](https://www.zynova.io/blog/ai-trends-2025?utm_source=openai)  
- [Global Biz Outlook - AI Trends](https://globalbizoutlook.com/top-10-ai-agent-trends-and-forecasts-to-watch/?utm_source=openai)  
- [Treu Partners - Technology Trends of 2025](https://treupartners.com/top-technology-trends-of-2025-ai-and-intelligent-ai-agents-lead/?utm_source=openai)  
- [Dev.to - AI Agent Trends](https://dev.to/aniruddhaadak/the-top-5-ai-agent-trends-for-2025-386g?utm_source=openai)  
- [Medium - No-Code AI Solutions](https://medium.com/coinmonks/top-10-ai-agent-development-trends-to-watch-in-2025-921f14272843?utm_source=openai)  
